In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile

In [2]:
with ZipFile('../data/train.csv.zip') as z:
    with z.open('train.csv') as f:
        train_df = pd.read_csv(f)

In [3]:
titles = train_df['title'].copy()

In [4]:
descriptions = train_df['description'].copy()

In [5]:
y = train_df['deal_probability'].copy()

In [6]:
del train_df

In [7]:
titles = titles.astype(str)

In [8]:
descriptions = descriptions.astype(str)

In [9]:
titles_descriptions = titles + descriptions
del titles, descriptions

In [10]:
titles_descriptions = titles_descriptions.tolist()

In [11]:
from keras.preprocessing.text import text_to_word_sequence

C:\Data\Soft\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer

In [13]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(titles_descriptions)
list_tokenized = tokenizer.texts_to_sequences(titles_descriptions)

In [14]:
from keras.preprocessing.sequence import pad_sequences

In [15]:
maxlen = 500
train = pad_sequences(list_tokenized, maxlen=maxlen)

In [16]:
import keras
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [28]:
embed_size = 128
inp = Input(shape=(maxlen, )) #maxlen=200 as defined earlier
x = Embedding(max_features, embed_size)(inp)
x = LSTM(60, return_sequences=True,name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="relu")(x)

In [29]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['mean_squared_error'])

In [30]:
batch_size = 128
epochs = 2
model.fit(train, y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 1353081 samples, validate on 150343 samples
Epoch 1/2
1353081/1353081 [==============================] - 7749s 6ms/step - loss: 0.0552 - mean_squared_error: 0.0552 - val_loss: 0.0540 - val_mean_squared_error: 0.0540
Epoch 2/2
1353081/1353081 [==============================] - 7948s 6ms/step - loss: 0.0528 - mean_squared_error: 0.0528 - val_loss: 0.0539 - val_mean_squared_error: 0.0539


In [80]:
with ZipFile('../data/test.csv.zip') as z:
    with z.open('test.csv') as f:
        test_df = pd.read_csv(f)

In [32]:
titles_test = test_df['title'].copy()

In [33]:
descriptions_test = test_df['description'].copy()

In [34]:
del test_df

In [35]:
titles_test = titles_test.astype(str)

In [36]:
descriptions_test = descriptions_test.astype(str)

In [ ]:
titles_descriptions_test = titles_test + descriptions_test

In [38]:
del titles_test, descriptions_test

In [39]:
titles_descriptions_test = titles_descriptions_test.tolist()

In [40]:
list_tokenized_test = tokenizer.texts_to_sequences(titles_descriptions_test)

In [78]:
maxlen = 500
test = pad_sequences(list_tokenized_test, maxlen=maxlen)

(508438, 500)


In [81]:
create_submission(model.predict(test), '../submissions/', 'first_nn', test_df)

## TODO token normalization

In [15]:
vocab = set()
for text in titles_descriptions:
    vocab |= set(text_to_word_sequence(text))

In [16]:
from pymorphy2 import MorphAnalyzer

In [17]:
ma = MorphAnalyzer()

In [18]:
normal = {}

In [ ]:
for word in vocab:
    normal[word] = ma.normal_forms(word)[0]